In [8]:
%pylab inline

import numpy as np
import pandas as pd

import csv
import json

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Introduction

* [Demo website](https://demo.deeppavlov.ai/#/en/textqa)
* [SQuAD 2.0 dataset](https://rajpurkar.github.io/SQuAD-explorer/)

Question Answering model은 주어진 정보안에서 정보를 찾아서 답변을 하는 NLP 모델입니다.<br>
다들 알다시피, OpenAI-GPT, ELMo, BERT, XLNet등등의 pre-trained model의 등장으로, <br>
NLP는 그 어느때보다도 빠른 변화를 맞고 있습니다. 


## Open-domain QA vs Closed-domin QA 

### Open-domain QA
**Open-domain QA**는 factoid questions에 한해서 다양한 분야의 모든 질문들을 포함하고 있습니다. <br>
대표적인 알고리즘은 페이스북에서 2017년에 만든 [DrQA](https://github.com/facebookresearch/DrQA/)가 있습니다. <br>
DrQA는 위키피디아의 정보를 활용해서 답변을 합니다.<br>
포인트는 위키피디아인데, 아시다시피 위키피디아는 정말 다양한 분야의 지식을 커버하고 있습니다. <br>
그런 이유로 open-domain 이라고 보시면 됩니다.


### Closed-domain QA
**Closed-domain QA**는 특정 도메인 (예. 약품, 자전거) 의 질문들을 다루며, <br>
당연히 특정 도메일의 지식만을 갖고서 답변을 합니다.<br>
DrQA와 동일하게, CDQA의 경우도 일반적으로 Retriever 와 Reader로 이루어 져 있습니다. <br>

가장 손쉽게 Retreiver를 만드는 방법은 question에 대한 tf-idf vector 그리고, 각 articles에 대한 tf-idf vectors 에 대해서 <br>
cosine similarity등으로 유사도를 측정해서 가장 높은 유사도 점수를 받은 articles을 뽑아냅니다. <br>
(이때 Tf-IDF로 만들때 uni-grams 또는 bi-grams 등을 기본 베이스로 사용할 수 있습니다.) 

 - uni-grams: I love you -> ("I", "love", "you")
 - bi-grams : I love you -> (("I", "love"), ("love", "you")) 즉.. 두단어씩 잡음
 
이후 documents를 paragraphs로 나눈뒤 (문서 하나 전체는 너무 크니까?), question과 함께 Reader의 input으로 보냅니다.<br>
Reader는 기본적으로 Bert같은 pre-trained model을 사용합니다.

- [How to create your own Question-Answering system easily with python 참고](https://towardsdatascience.com/how-to-create-your-own-question-answering-system-easily-with-python-2ef8abc8eb5)
 


In [6]:
from nltk import word_tokenize
from nltk.util import ngrams

tokens = word_tokenize('I love you')
bigrams = list(ngrams(tokens, 2))

print('tokens :', tokens)
print('bigrams:', bigrams)

tokens : ['I', 'love', 'you']
bigrams: [('I', 'love'), ('love', 'you')]


# DrQA

 - [Reading Wikipedia to Answer Open-Domain Questions 페이퍼](https://arxiv.org/pdf/1704.00051.pdf)
 
2017년 4월에 나온 논문으로 open domain question answering 문제를 다루고 있습니다.<br>
아래의 내용은 논문의 내용을 정리 하였습니다.


## Architecture of DrQA

DrQA 는 두가지로 이루어져 있습니다. 

1. **Document Retreiver**: 질문(Question)이 주어졌을때, bigram hasing 그리고 TF-IDF를 사용하여 관련된 문서들을(예. 위키피디아 문서들) 리턴
2. **Document Reader**: multi-layer recurrent neural network로 이루어진 machine comprehension model로서 answer spans을 관련된 문서들 사이에서 찾아냅니다. (answer span은 해당 문서에서의 정확한 위치를 나타냅니다.) 


> <span style="color:#777777">마치 vision object dectection에서 region proposal network가 이미지 상에서의 대략적인 객체 위치를 알려주고, RoI Pooling layer 그리고  classification 모델에서 정확하게 위치와 분류를 찾아내는 것 같은 느낌이네요. </span>

## Document Retreiver 

가장 중요한것은 몇백만건 이상의 문서중에서 가장 관련된 문서를 뽑아내는 것입니다. <br>
연산량은 단순할수록 좋기 때문에 ML을 사용해서 ranking을 하거나 하지 않으며, 단순하게 `inverted index lookup` 이후 `term vector model scroing` 등이 ElasticSearch를 사용한 위키피디아 검색보다 더 좋은 결과를 내놓습니다. 

> <span style="color:#777777"> **Inverted index**: 예를 들어서 특정 단어와, 문서번호, 문서안에서의 위치를 inverted index로 만들면 다음과 같습니다.</span> <br>
>> ```{"hello"   : [(1, 1)], 
>>   "everyone": [(1, 2)],
>>   "is"      : [(2, 4), (3, 2)]}```<br>
>> hello 라는 단어는 1번문서, 첫번째 단어에 존재하고, everyone은 첫번째 문서에서 2번째 단어에 속합니다. <br>
>> is 의 경우는 2번째 문서에서 4번째 단어, 3번째 문서에서 2번째 단어에 존재합니다. 

**페이퍼에서는 Articles과 Questions의 유사도를 찾는 방법으로 TF-IDF weighted bag-of word vectors 를 사용했습니다.**<br>
뭐 참 일반적이네요. ㅎㅎ 하지만 페이퍼에서는 좀 더 나아간 것이 있었으니 바로 단어의 순서를 **n-gram 을 사용**해서 고려했습니다. <br>
논문에서는 **bigram counts을 $ 2^{24} $ bins에 맴핑시키는 [unsigned murmur3 hash](https://alex.smola.org/papers/2009/Weinbergeretal09.pdf) 기법을 사용**했습니다.


- <span style="color:#777777">[MurMurHash Python 참고](https://pypi.org/project/mmh3/)</span>
- <span style="color:#777777">MurMurHash는 non-cryptographic hash function으로서 일반적인 해쉬를 사용할때 좋지만, reverse하기 쉽습니다.</span>

페이퍼상에서 Document Retreiver는 각 질문하나당 5개의 articles을 리턴해주었습니다.



## Document Reader

### 용어정리

**A Question** 
- $ (\text{a single question})\ q \supset \{q_1, q_2, ..., q_l\} $
- 특정 하나의 Question $ q $ 는 l tokens으로 이루어져 있습니다.
- 즉 $ q_1 $ 은 특정 질문 속에 있는 첫번째 단어 (token)
- a question q consisting of l tokens $ {q_1, . . . , q_l} $

**A Paragraph**

- $ (\text{a single paragraph})\ p \supset \{p_1, p_2, ..., p_m\} $
- 특정 하나의 Paragraph는 m개의 tokens으로 이루어져 있습니다.
-   a single paragraph p consists of m tokens $ {p_1, . . . , p_m} $

### Paragraph Encoding

먼저 특정 paragraph안에 있는 모든 tokens $ p_i $ 를 feature vectors $ \mathbf{\vec{p}}_i \in \mathbb{R}^d $ 로 변환시킵니다.<br>
이후 Bidirectional LSTM에 input 으로 집어넣습니다.

- a paragraph: $ p $ (문서 안에 있는 문장 하나) 
- a token in a paragraph $ p $ : $ p_i $ (문장안에 있는 단어 하나)
- a sequence of feature vectors in a paragraph: $ \mathbf{\vec{p}}_i \in \mathbb{R}^d $ (문장안의 단어를  -> embedding vector로 변환 시킨것) 


$$ \{\mathbf{p}_1, \mathbf{p}_2, ..., \mathbf{p}_m\} = \text{BiLSTM}( \{ \mathbf{\vec{p}}_1, \mathbf{\vec{p}}_2, ..., \mathbf{\vec{p}}_m \}) $$


BiLSTM함수안에 들어가는 것은, 하나의 paragraph안의 embedding vectors가 sequence형태로 들어가는 것

### Word Embedding 

최근 트렌드 (2020년 1월 기준)는 Transformer를 많이 사용하는 추세이나, 논문에서는 Glove를 사용하였습니다. 

$$ f_{emb}(p_i) = \mathbf{E}(p_i) $$

 - **300-dimensional Glove word embeddings** 사용하였으며, 840B Web crawl data를 사용해서 학습되었음. 
 - Embedding Layer는 **freeze시켰지만, 1000개의 가장 빈번하게 나오는 단어들에 대해서는 fine-tune**을 하였음 <br>(what, how, which, many 같은 단어의 경우 QA system에서 중요하기 때문에 fine-tune 시킴) 


### Token Features

단순히 단어만 feature input으로 넣은게 아니라 아래 3개의 features 더 추가했습니다. 

$$ f_{token}(p_i) = (\mathbf{POS}(p_i), \mathbf{NER}(p_i), \mathbf{TF}(p_i)) $$

- $ \mathbf{POS} $ : part-of-speech 
- $ \mathbf{NER} $ : Named entity recognition tags 
- $ \mathbf{TF} $  : (normalized) Term Frequency

In [14]:
import spacy

space_ = spacy.load('en', parser=False)
text = space_('Kakao is South Korean internet company that was '
              'established by Kim BumSoo in 2010')

df = []
for word in text:
    df.append([word, word.tag_, word.ent_type_])
    
pd.DataFrame(df, columns=['word', 'pos', 'ner'])

,word,pos,ner
0,Kakao,NNP,GPE
1,is,VBZ,
2,South,JJ,NORP
3,Korean,JJ,NORP
4,internet,NN,
5,company,NN,
6,that,WDT,
7,was,VBD,
8,established,VBN,
9,by,IN,


### Aligned Question Embedding

$$ f_{align}(p_i) = \sum_j a_{i, j} \mathbf{E}(q_j) $$

- $ a_{i, j} $ : Attention score
- $ \mathbf{E}(q_j) $ : 질문 $ q_j $ 를 word embedding vector로 변환시킨 것
- a paragraph $ p_i $ 와 각각의 질문들 $ q_j $ 사이의 유사도 (similarity)를 구합니다.

Attention $ a_{i, j} $ 는 다음과 같은 함수를 통해서 학습을 하게 됩니다.

$$ a_{i, j} = \frac{exp\left( \alpha(\mathbf{E}(p_i))  \cdot \alpha(\mathbf{E}(q_i))  \right)}
{\sum_j \alpha(\mathbf{E}(p_i))  \cdot \alpha(\mathbf{E}(q_i))} $$

 - $ \alpha(\cdot) $ : single dense layer with ReLU nonlinearity
 - word embeddings 끼리는 dot product를 한다
 - **soft alignment**: exact match와 결이 좀 다르게, 유사하지만 전혀 다른 단어들을 잡아내는데 도움을 줍니다. <br>(예를 들어서 자동차 그리고 바퀴는 서로 전혀 다른 단어이지만, 밀접한 연관성은 갖고 있습니다. )

### Question Encoding 






## Prediction

최종적으로는 맞는 답이라고 판단되는 문장속의 범위 (span of tokens)를 예측합니다. <br>
Input 으로 다음과 같습니다.

 - paragraph vectors: $ \{\mathbf{p}_1, \mathbf{p}_2, ..., \mathbf{p}_m\} $ 여기서 $ \mathbf{p}_i $ 는 단어 하나를 뜻하는 embedding vector 입니다.
 - question vector: $ \mathbf{q} $

각 Input마다 독립적인 2개의 classifiers를 만드며,<br>
해당 classifiers는 어디 지점에서 시작하는지, 그리고 어디 지점에서 끝나는지 확률을 구합니다.

$$ \begin{align} P_{start}(i) &\propto exp(\mathbf{p}_i \mathbf{W}_s \mathbf{q}) \\
P_{end}(j) &\propto exp(\mathbf{p}_j \mathbf{W}_s \mathbf{q})
\end{align} $$

모든 시작점과 끝점의 확률값이 구해졌으면 $ i \leq j \leq i+15 $ 범위를 갖는 것에서 (즉 시작점에서 +15단어 범위 안에서), <br>
$ P_{start}(i) x P_{end}(j) $의 값을 maximize시키는 값을 구합니다.

# SQuAD 2.0 Dataset

**S**tanford **Qu**estion **A**nswering **D**ataset (SQuAD) 데이터는 위키피디아를 기반으로한 질문 그리고 답변이 있는 데이터셋입니다.

In [13]:
!wget -N -q https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json train-v2.0.json
!wget -N -q https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json dev-v2.0.json

In [33]:

with open('dev-v2.0.json', 'rt') as f:
    dataset = json.load(f)
    
with open('train.json', 'w') as f:
    writer = csv.writer(f)
    for article in dataset['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [a['text'] for a in qa['answers']]
                starts = [a['text'] for a in qa['answers']]
                f.write(json.dumps({'question': question, 'answer': answers, 'start': starts}))
                f.write('\n')


In [46]:
with open('train.json', 'w') as f:
    writer = csv.writer(f, delimiter='')
    writer.writerow(['a', 'b'])
    
!cat train.json

a	b


In [47]:
dataset.keys()
dataset['data'][0]['paragraphs'][0]


{'qas': [{'question': 'In what country is Normandy located?',
   'id': '56ddde6b9a695914005b9628',
   'answers': [{'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159},
    {'text': 'France', 'answer_start': 159}],
   'is_impossible': False},
  {'question': 'When were the Normans in Normandy?',
   'id': '56ddde6b9a695914005b9629',
   'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
    {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
    {'text': '10th and 11th centuries', 'answer_start': 94},
    {'text': '10th and 11th centuries', 'answer_start': 94}],
   'is_impossible': False},
  {'question': 'From which countries did the Norse originate?',
   'id': '56ddde6b9a695914005b962a',
   'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
    {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
    {'text': 'Denmark, Iceland and Norway', 'answer_star